In [4]:
from pytube import YouTube
import pandas as pd
from moviepy.editor import *
import os
import pytube
from pytube import YouTube
from pytube.exceptions import VideoUnavailable
from pytube.exceptions import VideoPrivate, AgeRestrictedError,LiveStreamError, RecordingUnavailable, MembersOnly, VideoRegionBlocked
import pandas as pd
import numpy as np
import pytube.exceptions
import matplotlib.pyplot as plt
from google.cloud import storage
import re

In [2]:
#getting the csv file and dropping videos, which end_time is more than 300
def get_data(x):
    #reading the csv
    df = pd.read_csv("/Users/philkolling/Downloads/kinetics700_2020/train.csv")
    df =df.copy().head(x)
    #dropping the videos, which end_time is higher than 300
    df =df.drop(df.loc[df['time_end']>300].index).reset_index(drop=True)
    return df

In [63]:
#Test df for test_model



In [133]:
def remove_long_videos(df):
    test = df.copy()
    #count is just for testing
    #count = 0
    #list of all the lengths in order to build the new column
    lengths = []
    #iterating over through all the videos
    for video in range(0,len(test)):
        url = "https://www.youtube.com/watch?v="+test.youtube_id[video]
        yt = YouTube(url)
        #handling error messages
        try:
            stream = yt.streams
        except (KeyError,VideoPrivate,VideoUnavailable,AgeRestrictedError,LiveStreamError, RecordingUnavailable, MembersOnly, VideoRegionBlocked):
            test.drop(video,inplace=True)
            continue
        #drop video if longer than 5min
        if yt.length > 300:
            test.drop(video,inplace=True)
        #if video is already clipped, but end_time is higher than lenght of the video, we set new start and end points
        elif yt.length == 10:
            test.time_start[video] = 0
            test.time_end[video] = yt.length
            length = yt.length
            lengths.append(length)
        #if time_end greater than the length and it's not clipped we drop it
        elif yt.length < test.time_end[video]:
            test.drop(video,inplace=True)
        #appending the lengths to the list     
        else:
            length = yt.length
            lengths.append(length)
        #count = count +1
        #print(count)
    #creating the new column    
    test["lenghts"] = lengths
    #returning the new dataframe
    return test.reset_index(drop=True)        
                
        

In [5]:
def get_clean_data(x):
    df = get_data(x)
    df = remove_long_videos(df)
    return df

In [9]:
#tmp=get_clean_data(20)
tmp

,label,youtube_id,time_start,time_end,split,lenghts
0,clay pottery making,---0dWlqevI,19,29,train,30
1,news anchoring,---aQ-tA5_A,9,19,train,206
2,using bagging machine,---j12rm3WI,14,24,train,153
3,javelin throw,--07WQ2iBlw,0,10,train,10
4,climbing a rope,--0NTAs-fA0,29,39,train,39
5,sipping cup,--0l35AkU34,68,78,train,90
6,flipping pancake,--33Lscn6sk,0,10,train,10
7,eating spaghetti,--3ouPhoy2A,20,30,train,43
8,dribbling basketball,--4-0ihtnBU,58,68,train,156
9,playing tennis,--56QUhyDQM,185,195,train,205


In [5]:
#download youtube video to folder, filename= youtube_id
path = "/Users/philkolling/Downloads/YoutubeVideos" #path has to be edited
path_test = "/Users/philkolling/Downloads/EditedVideos" #path_test has to be edited
client = storage.Client.from_service_account_json("/Users/philkolling/code/philkolling/gcp/project/peppy-webbing-332911-743c3173bc03.json")
bucket = client.get_bucket('737-human-action-recognition-bucket')
def videoDownload(df): 
    URL = "https://www.youtube.com/watch?v="
    for i in range(0,len(df)):
        # creating the URL link
        video = YouTube(URL + df.youtube_id[i])
        #downloading the video in a local folder, name of the video is label/youtube_id
        video.streams.get_by_itag(18).download(path,filename=f"{df.label[i]}_{df.youtube_id[i]}.mp4")
        
        #creating the path to the video to clip it
        source_path = os.path.join(path,f'{df.label[i]}_{df.youtube_id[i]}.mp4')
        clip = VideoFileClip(source_path)
        #defining the frames per second
        fps = 10
        #creating the subclips with start and end from the csv
        subclip =clip.subclip(df.time_start[i],df.time_end[i])
        #cutting out the audio
        subclip = subclip.without_audio()
        
        #save the cliped file to another folder 
        output_path = os.path.join(path_test,f'{df.label[i]}_{df.youtube_id[i]}.mp4')
        subclip.write_videofile(output_path,fps)
        
        #remove original youtube video
        os.remove(f"{path}/{df.label[i]}_{df.youtube_id[i]}.mp4")
        
        #getting the actual filename we want to upload
        file = os.listdir(f'{path_test}')[0]
        if file != ".ipynb_checkpoints":
            #cretinga blob in order to upload it to the right folder based on the label of the video
            blob = bucket.blob(f'train_data/{df.label[i]}/{file}')
            blob.upload_from_filename(f"{path_test}/{file}")
        else:
            continue
        #remove the cliped video from the local computer
        os.remove(f"{path_test}/{df.label[i]}_{df.youtube_id[i]}.mp4")
        
        
        
        
        

In [32]:
#videoDownload(tmp)

In [ ]:

    #take youtube video from local file and cut it to a 10sec video to upload it to gcp
    #Steps:
        #download youtube video with clean df
        #save in local directory
        #load file from local directory
        #cut video with movie.py to 10sec
        #upload to gcp in the right folder based on label
    #Questions:
        #filename?
        #tell the computer which folder it should take

# Test Model

In [95]:
test_model_df = pd.read_csv("/Users/philkolling/Downloads/kinetics700_2020/train.csv")
test_model_df.head()

,label,youtube_id,time_start,time_end,split
0,clay pottery making,---0dWlqevI,19,29,train
1,news anchoring,---aQ-tA5_A,9,19,train
2,using bagging machine,---j12rm3WI,14,24,train
3,javelin throw,--07WQ2iBlw,1,11,train
4,climbing a rope,--0NTAs-fA0,29,39,train


In [96]:
test_model_df.shape

(544823, 5)

In [97]:
test_model_df = test_model_df.drop(test_model_df.loc[test_model_df['time_end']>40].index).reset_index(drop=True)

In [98]:
test_model_df.shape

(273989, 5)

In [99]:
test_model_df["label"].value_counts()

long jump          872
triple jump        865
front raises       861
finger snapping    848
faceplanting       845
                  ... 
flint knapping      67
making slime        66
playing cards       61
calculating         54
fly tying           37
Name: label, Length: 700, dtype: int64

In [100]:
frames = []
test_labels = ['walking on stilts', 'silent disco', 'being excited',
       'pretending to be a statue', 'walking with crutches',
       'slicing onion', 'stacking cups', 'making latte art',
       'playing checkers', 'using megaphone']

for i in test_labels:
    g = test_model_df[test_model_df.label == i].sample(50)
    frames.append(g)

equally_sampled = pd.concat(frames)
equally_sampled = equally_sampled.reset_index(drop=True)

In [102]:
equally_sampled.shape

(500, 5)

In [12]:
#test_model_df = remove_long_videos(equally_sampled)

In [104]:
test_model_df["label"].value_counts()

walking on stilts            44
pretending to be a statue    42
making latte art             42
slicing onion                41
silent disco                 40
playing checkers             37
using megaphone              36
being excited                35
walking with crutches        35
stacking cups                33
Name: label, dtype: int64

In [105]:
test_model_df.shape

(385, 6)

In [11]:
#videoDownload(test_model_df.tail(300).reset_index(drop=True))

In [10]:
#test_model_df[test_model_df["label"]!="beeing exited","silent disco","walking on stilts"]

In [117]:
frames = []
test_labels = [
       'pretending to be a statue', 'walking with crutches',
       'slicing onion', 'stacking cups', 'making latte art',
       'playing checkers', 'using megaphone']

for i in test_labels:
    g = test_model_df[test_model_df.label == i].sample(33)
    frames.append(g)

equally_sampled = pd.concat(frames)
test_model_df = equally_sampled.reset_index(drop=True)

In [118]:
test_model_df

,label,youtube_id,time_start,time_end,split,lenghts
0,pretending to be a statue,V6yoIspvHNo,10,20,train,66
1,pretending to be a statue,oDGcRv7pNDQ,0,10,train,15
2,pretending to be a statue,G2QLRu_vYIM,10,20,train,20
3,pretending to be a statue,_JRfRdNs3dg,3,13,train,15
4,pretending to be a statue,az2TD0XRvAg,16,26,train,78
...,...,...,...,...,...,...
226,using megaphone,yarYeKq_CKo,5,15,train,17
227,using megaphone,yZC3zuUiaSc,0,10,train,13
228,using megaphone,C8YHndsvZo8,19,29,train,90
229,using megaphone,J24fakyZPNI,16,26,train,53


In [9]:
#videoDownload(test_model_df)

In [124]:
temporary = len(os.listdir("/Users/philkolling/Downloads/EditedVideos"))
temporary

353

In [138]:
phil_df = pd.read_csv("/Users/philkolling/Downloads/train_reduced.csv")

In [139]:
labels = phil_df["label"].unique()[175:350]

In [140]:
phil_df = phil_df[phil_df["label"].isin(labels)]

In [141]:
phil_df = phil_df.reset_index(drop=True)

In [142]:
%%time
phil_df_clean = remove_long_videos(phil_df)

CPU times: user 2h 10min 33s, sys: 2min 57s, total: 2h 13min 31s
Wall time: 6h 5min 27s


In [143]:
phil_df_clean.to_csv("/Users/philkolling/Downloads/reduced_csv_175_350.csv")

In [145]:
phil_df_clean.shape

(14669, 6)

In [148]:
phil_df_clean2 = phil_df_clean.reset_index(drop=True)

In [8]:
#videoDownload(phil_df_clean2)

In [160]:
phil_df_clean2["label"].unique()

array(['ski ballet', 'opening present', 'celebrating', 'bartending',
       'washing hands', 'polishing metal', 'ice fishing',
       'shoveling snow', 'running on treadmill', 'climbing ladder',
       'dancing charleston', 'dancing ballet', 'playing polo',
       'rolling pastry', 'hitting baseball', 'whistling',
       'blowing out candles', 'alligator wrestling', 'crying',
       'juggling soccer ball', 'watching tv',
       'decorating the christmas tree', 'hopscotch', 'headbutting',
       'roasting pig', 'headbanging', 'riding scooter',
       'putting on mascara', 'catching or throwing softball',
       'massaging feet', 'trapezing', 'dodgeball', 'waxing legs',
       'smoking hookah', 'licking', 'luge', 'laying concrete', 'burping',
       'archery', 'reading newspaper', 'trimming trees',
       'exercising with an exercise ball', 'cracking back',
       'jumpstyle dancing', 'roasting marshmallows', 'shearing sheep',
       'blowing bubble gum', 'playing volleyball', 'tasting f

In [163]:
labels2 = phil_df_clean2["label"].unique()[31:175]
labels2

array(['dodgeball', 'waxing legs', 'smoking hookah', 'licking', 'luge',
       'laying concrete', 'burping', 'archery', 'reading newspaper',
       'trimming trees', 'exercising with an exercise ball',
       'cracking back', 'jumpstyle dancing', 'roasting marshmallows',
       'shearing sheep', 'blowing bubble gum', 'playing volleyball',
       'tasting food', 'ice climbing', 'mowing lawn', 'sharpening knives',
       'blasting sand', 'shining shoes', 'lifting hat', 'spelunking',
       'using atm', 'carving ice', 'pole vault', 'golf driving',
       'casting fishing line', 'punching bag', 'sleeping',
       'using a wrench', 'petting horse', 'petting animal (not cat)',
       'tackling', 'tying shoe laces', 'stretching leg',
       'making a sandwich', 'playing saxophone', 'laying bricks',
       'playing beer pong', 'playing tennis', 'catching fish',
       'eating ice cream', 'playing ice hockey', 'unboxing',
       'chopping wood', 'pouring milk', 'hugging (not baby)',
       'thr

In [164]:
phil_df_clean3 = phil_df_clean2[phil_df_clean2["label"].isin(labels2)]

In [173]:
phil_df_clean3.head(13)

,label,youtube_id,time_start,time_end,split,lenghts
2642,dodgeball,VmMazKiGp8Y,17,27,train,111
2643,dodgeball,Q3zs8esNURs,46,56,train,80
2644,dodgeball,QibDiBjFn_Q,38,48,train,54
2645,dodgeball,Z4sUA69lHF8,20,30,train,30
2646,dodgeball,J3OG-RLowJc,29,39,train,39
2647,dodgeball,6zE7M5CPh1s,19,29,train,68
2648,dodgeball,O-qn7sosLn4,3,13,train,174
2649,dodgeball,Ei39qrnIOt8,20,30,train,99
2650,dodgeball,sqzrKKbSfIg,22,32,train,39
2651,dodgeball,266DF3P2lt0,42,52,train,55


In [177]:
phil_df_clean4 = phil_df_clean3[13:].reset_index(drop=True)

In [7]:
#videoDownload(phil_df_clean4)

In [180]:
labels3 = phil_df_clean4["label"].unique()[11:175]
labels3

array(['cracking back', 'jumpstyle dancing', 'roasting marshmallows',
       'shearing sheep', 'blowing bubble gum', 'playing volleyball',
       'tasting food', 'ice climbing', 'mowing lawn', 'sharpening knives',
       'blasting sand', 'shining shoes', 'lifting hat', 'spelunking',
       'using atm', 'carving ice', 'pole vault', 'golf driving',
       'casting fishing line', 'punching bag', 'sleeping',
       'using a wrench', 'petting horse', 'petting animal (not cat)',
       'tackling', 'tying shoe laces', 'stretching leg',
       'making a sandwich', 'playing saxophone', 'laying bricks',
       'playing beer pong', 'playing tennis', 'catching fish',
       'eating ice cream', 'playing ice hockey', 'unboxing',
       'chopping wood', 'pouring milk', 'hugging (not baby)',
       'throwing tantrum', 'playing bagpipes', 'eating hotdog', 'drawing',
       'drop kicking', 'ironing', 'docking boat', 'waxing back',
       'sticking tongue out', 'setting table', 'tightrope walking',
     

In [193]:
phil_df_clean5 = phil_df_clean4[phil_df_clean4["label"].isin(labels3)][28:]

In [195]:
phil_df_clean5 = phil_df_clean5.reset_index(drop=True)

In [6]:
#videoDownload(phil_df_clean5)

In [200]:
labels4 = phil_df_clean5["label"].unique()[17:]

In [213]:
phil_df_clean6 = phil_df_clean5[phil_df_clean5["label"].isin(labels4)][12:]

In [214]:
phil_df_clean6 = phil_df_clean6[phil_df_clean6["lenghts"] <= 90]

In [216]:
phil_df_clean7 = phil_df_clean6.reset_index(drop=True)

In [218]:
phil_df_clean7

,label,youtube_id,time_start,time_end,split,lenghts
0,golf driving,VFTWNtWfIKg,0,10,train,27
1,golf driving,gv5nr0EDwXM,41,51,train,51
2,golf driving,dionWxR5NJc,5,15,train,17
3,golf driving,PC_rUcAndHg,1,11,train,16
4,golf driving,vhGIZ5steZE,3,13,train,18
...,...,...,...,...,...,...
6737,riding snow blower,_yXS216fw5U,58,68,train,87
6738,riding snow blower,8GHhjyuMAqw,0,10,train,13
6739,riding snow blower,PSdXsFtQk7o,8,18,train,40
6740,riding snow blower,DKfIkE4LKvU,6,16,train,70


In [5]:
#videoDownload(phil_df_clean7)

In [223]:
labels7 = phil_df_clean7["label"].unique()[99:]
labels7

array(['jumping into pool', 'filling eyebrows', 'sharpening pencil',
       'playing harp', 'mushroom foraging', 'skiing mono',
       'swinging baseball bat', 'throwing snowballs', 'playing violin',
       'cooking scallops', 'riding mule', 'playing accordion', 'shot put',
       'kitesurfing', 'playing piano', 'snorkeling', 'riding snow blower'],
      dtype=object)

In [240]:
phil_df_clean8 = phil_df_clean7[phil_df_clean7["label"].isin(labels7)][24:]

In [242]:
phil_df_clean9 = phil_df_clean8.reset_index(drop=True)

In [4]:
#videoDownload(phil_df_clean9)

In [4]:
julian_df = pd.read_csv("/Users/philkolling/Downloads/df_last_third.csv")

In [3]:
#videoDownload(julian_df)

In [12]:
julian_label = julian_df["label"].unique()[18:]
julian_label

array(['faceplanting', 'hugging baby', 'swinging on something',
       'high kick', 'cleaning toilet', 'tying knot (not on a tie)',
       'opening door',
       'throwing ball (not baseball or American football)',
       'jumping bicycle', 'parasailing', 'hand washing clothes',
       'making tea', "massaging person's head", 'winking',
       'eating doughnuts', 'fly tying', 'bench pressing', 'longboarding',
       'putting on eyeliner', 'twiddling fingers', 'welding',
       'jumping sofa', 'changing oil', 'swimming butterfly stroke',
       'feeding fish', 'sanding floor', 'cutting orange',
       'scrambling eggs', 'tagging graffiti', 'bee keeping',
       'shooting off fireworks', 'playing poker', 'bending back', 'situp',
       'making sushi', 'baby waking up', 'sword fighting'], dtype=object)

In [20]:
julian_df.head()

,Unnamed: 0,label,youtube_id,time_start,time_end,split,lenghts
0,0,waking up,fVXHY4671lc,10,20,train,24
1,1,waking up,Zb41Ia9_L-o,0,10,train,23
2,2,waking up,EZA-FEjFnpI,1,11,train,67
3,3,waking up,buJK8FauuN0,9,19,train,21
4,4,waking up,awL-P4dbF_Y,31,41,train,42


In [33]:
julian_df_2 = julian_df[julian_df["label"].isin(julian_label)][53:]

In [34]:
julian_df_2

,Unnamed: 0,label,youtube_id,time_start,time_end,split,lenghts
1155,1155,faceplanting,o-y--WgmldY,6,16,train,30
1156,1156,faceplanting,_loSLLC5DFE,10,20,train,21
1157,1157,faceplanting,sr-VLuQUsQw,21,31,train,56
1158,1158,faceplanting,SfnVv-JRfa0,21,31,train,31
1159,1159,faceplanting,icweOqQwPFY,10,20,train,85
...,...,...,...,...,...,...,...
3326,3326,sword fighting,7RYMdgM4Dx0,5,15,train,54
3327,3327,sword fighting,gEky8kQ8COg,24,34,train,42
3328,3328,sword fighting,H6QkI2SMcRY,9,19,train,65
3329,3329,sword fighting,PQCLuMSRFWU,20,30,train,68


In [37]:
julian_df3 = julian_df_2.drop(columns=["Unnamed: 0"]).reset_index(drop=True)

In [38]:
julian_df3.head(20)

,label,youtube_id,time_start,time_end,split,lenghts
0,faceplanting,o-y--WgmldY,6,16,train,30
1,faceplanting,_loSLLC5DFE,10,20,train,21
2,faceplanting,sr-VLuQUsQw,21,31,train,56
3,faceplanting,SfnVv-JRfa0,21,31,train,31
4,faceplanting,icweOqQwPFY,10,20,train,85
5,hugging baby,9Ap3cL4Lp4s,18,28,train,37
6,hugging baby,z2LNzpfECi8,2,12,train,15
7,hugging baby,8bj5W3ZdS-k,0,10,train,30
8,hugging baby,e9ZDqmW7FIg,0,10,train,65
9,hugging baby,Jd-b2AWC7pk,4,14,train,17


In [2]:
#videoDownload(julian_df3)

In [42]:
hello = "phiL"

In [13]:
def get_bucket_numbers():
    client = storage.Client.from_service_account_json("/Users/philkolling/code/philkolling/gcp/project/peppy-webbing-332911-743c3173bc03.json")
    bucket = client.get_bucket('737-human-action-recognition-bucket')

    blobs = bucket.list_blobs(prefix='train_data/')

    videos = {}

    for blob in blobs:
        if blob.name.endswith('/'):
            continue

        video = re.sub(r'train_data/.*/','', blob.name)
        video = re.sub(r'_.*.mp4','', video)
        if video in videos:
            videos[video] += 1
        else:
            videos[video] = 0

    videos.pop('.DS_Store')

    for key,value in videos.items():
         (f'{key}:{value}')
    return videos

In [14]:
label_numbers = get_bucket_numbers()

In [44]:
label_numbers_df = pd.DataFrame.from_dict(label_numbers, orient='index')

In [54]:
tmp45 = label_numbers_df.rename(columns={"values" : "0"})

In [56]:
tmp45

,0
abseiling,6
acting in play,79
tango dancing,245
adjusting glasses,64
air drumming,91
...,...
writing,74
yarn spinning,94
yawning,35
yoga,84


In [50]:
label_numbers_df.sort_values(by ="values")

KeyError: 'values'